In [0]:
# from google.colab import files
# uploaded = files.upload()

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')  

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# ! ls '/content/gdrive/My Drive/celeba/data/celeba/images' | wc -l

In [0]:
data_path = "/content/gdrive/My Drive/nlp_project/english_hindi/"
train_name = "train_new_demojise_large.txt"
validation_name = "validation_new_demojise.txt"
model_save_name = "hinglish_model_1layer_0.5dropout.txt"

# data_path = "/content/gdrive/My Drive/nlp_project/spanglish/"
# train_name = "spanglish_train_demojised.txt"
# validation_name = "spanglish_validation_demojised.txt"
# model_save_name = "spanglish_model.txt"
bert_model = "bert-base-multilingual-uncased"

In [0]:
import sys
sys.path.append(data_path)


In [7]:
import torch

import random
import numpy as np

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

!pip install transformers

     |████████████████████████████████| 450kB 3.5MB/s 
     |████████████████████████████████| 870kB 16.6MB/s 
     |████████████████████████████████| 1.0MB 22.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884629 sha256=bd1da97504cb15a516d16edb97b85d4886cd4ec8c00006a8e7936eee243e8eca
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [8]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained(bert_model)

In [9]:
len(tokenizer.vocab)
tokens = tokenizer.tokenize('how ARE you ')

print(tokens)
indexes = tokenizer.convert_tokens_to_ids(tokens)

print(indexes)


['how', 'are', 'you']
[12548, 10320, 10855]


In [10]:
init_token = tokenizer.cls_token
eos_token = tokenizer.sep_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token

print(init_token, eos_token, pad_token, unk_token)


# We can get the indexes of the special tokens by converting them using the vocabulary...

# In[7]:


init_token_idx = tokenizer.convert_tokens_to_ids(init_token)
eos_token_idx = tokenizer.convert_tokens_to_ids(eos_token)
pad_token_idx = tokenizer.convert_tokens_to_ids(pad_token)
unk_token_idx = tokenizer.convert_tokens_to_ids(unk_token)

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)


# ...or by explicitly getting them from the tokenizer.

# In[8]:


init_token_idx = tokenizer.cls_token_id
eos_token_idx = tokenizer.sep_token_id
pad_token_idx = tokenizer.pad_token_id
unk_token_idx = tokenizer.unk_token_id

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)


# Another thing we need to handle is that the model was trained on sequences with a defined maximum length - it does not know how to handle sequences longer than it has been trained on. We can get the maximum length of these input sizes by checking the `max_model_input_sizes` for the version of the transformer we want to use. In this case, it is 512 tokens.

# In[9]:


max_input_length = tokenizer.max_model_input_sizes[bert_model]

print(max_input_length)


# Previously we have used the `spaCy` tokenizer to tokenize our examples. However we now need to define a function that we will pass to our `TEXT` field that will handle all the tokenization for us. It will also cut down the number of tokens to a maximum length. Note that our maximum length is 2 less than the actual maximum length. This is because we need to append two tokens to each sequence, one to the start and one to the end.

# In[10]:


def tokenize_and_cut(sentence):
    tokens = tokenizer.tokenize(sentence) 
    tokens = tokens[:max_input_length-2]
    return tokens


# Now we define our fields. The transformer expects the batch dimension to be first, so we set `batch_first = True`. As we already have the vocabulary for our text, provided by the transformer we set `use_vocab = False` to tell torchtext that we'll be handling the vocabulary side of things. We pass our `tokenize_and_cut` function as the tokenizer. The `preprocessing` argument is a function that takes in the example after it has been tokenized, this is where we will convert the tokens to their indexes. Finally, we define the special tokens - making note that we are defining them to be their index value and not their string value, i.e. `100` instead of `[UNK]` This is because the sequences will already be converted into indexes.
# 
# We define the label field as before.

# In[87]:


from torchtext import data
data

[CLS] [SEP] [PAD] [UNK]
101 102 0 100
101 102 0 100
512


<module 'torchtext.data' from '/usr/local/lib/python3.6/dist-packages/torchtext/data/__init__.py'>

In [0]:
UID = data.Field(sequential=False, use_vocab=False, pad_token=None)
TEXT = data.Field(batch_first = True,
                  use_vocab = False,
                  tokenize = tokenize_and_cut,
                  preprocessing = tokenizer.convert_tokens_to_ids,
                  init_token = init_token_idx,
                  eos_token = eos_token_idx,
                  pad_token = pad_token_idx,
                  unk_token = unk_token_idx)

LABEL = data.LabelField()

# We load the data and create the validation splits as before.

# In[89]:


# from torchtext import datasets
# fields = {'uid':('uid',UID),'text': ('text', TEXT), 'label': ('label', LABEL) }
fields = [('uid',UID),('text', TEXT),('label', LABEL)]
train_data, test_data = data.TabularDataset.splits(
                                        path = data_path,
                                        train = train_name,
                                        test = validation_name,
                                        format = 'tsv',
                                        fields = fields,
                                        skip_header = True
)
# train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)
# valid_data = train_data
train_data, valid_data = train_data.split(0.8,random_state = random.seed(SEED))


# In[90]:


# print(vars(train_data[1]))
# print(tokenizer.convert_ids_to_tokens(vars(test_data[331])))


In [0]:
# for i in range(len(train_data)):
#   try:
#     x = (vars(train_data[i])['uid'],vars(train_data[i])['label'],vars(train_data[i])['text'])
#   except:
#     x = print(vars(train_data[i])['uid'],tokenizer.convert_ids_to_tokens(vars(train_data[i])['text']))

# for i in range(len(test_data)):
#   try:
#     x = print(vars(train_data[i])['uid'],vars(train_data[i])['label'],vars(train_data[i])['text'])
#   except:
#     x = print(vars(train_data[i])['uid'],tokenizer.convert_ids_to_tokens(vars(train_data[i])['text']))

# for i in range(len(valid_data)):
#   try:
#     x = (vars(train_data[i])['uid'],vars(train_data[i])['label'],vars(train_data[i])['text'])
#   except:
#     x = print(vars(train_data[i])['uid'],tokenizer.convert_ids_to_tokens(vars(train_data[i])['text']))

In [13]:
print(f"Number of training examples: {len(train_data)}")
print(f"Number of validation examples: {len(valid_data)}")
print(f"Number of testing examples: {len(test_data)}")


# We can check an example and ensure that the text has already been numericalized.

# In[92]:


# print(vars(train_data.examples[0])['label'])
print(vars(train_data.examples[1]))
print(vars(train_data.examples[2]))



# We can use the `convert_ids_to_tokens` to transform these indexes back into readable tokens.

# In[93]:


tokens = tokenizer.convert_ids_to_tokens(vars(train_data.examples[0])['text'])

print(tokens)


# Although we've handled the vocabulary for the text, we still need to build the vocabulary for the labels.

# In[94]:

UID.build_vocab(train_data)
LABEL.build_vocab(train_data)


# In[95]:


print(LABEL.vocab.stoi)
print(UID.vocab.stoi)

# As before, we create the iterators. Ideally we want to use the largest batch size that we can as I've found this gives the best results for transformers.

# In[96]:


BATCH_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


Number of training examples: 14303
Number of validation examples: 3576
Number of testing examples: 3000
{'uid': '17763', 'text': [91078, 10167, 55967, 10900, 59576, 11655, 14903, 10112, 65054, 10111, 90248, 18478], 'label': '0'}
{'uid': '40753', 'text': [16623, 29068, 32314, 22581, 62459, 10116, 16623, 29068, 32314, 18369, 10131, 17612, 11406, 65411, 12912, 32314, 12013, 10518, 10900, 12314, 11561, 11038, 10416, 21185, 11742, 11448, 65411, 119, 119, 10953, 10104, 16729, 10115, 10128], 'label': '1'}
['abe', 'maji', '##d', 'se', 'chap', '##al', 'chor', '##i', 'nahi', 'hot', '##i', 'jis', '##e', 'jar', '##urt', 'hot', '##i', 'he', 'wo', 'lek', '##ar', 'ja', '##ta', 'he']
defaultdict(<function _default_unk_index at 0x7f38b4f1c620>, {'1': 0, '2': 1, '0': 2})
defaultdict(<function _default_unk_index at 0x7f38b4f1c620>, {'<unk>': 0, '1002': 1, '10020': 2, '1003': 3, '1004': 4, '10043': 5, '10045': 6, '1005': 7, '10057': 8, '10071': 9, '10074': 10, '10075': 11, '10079': 12, '10089': 13, '10090

In [0]:
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    sort_key=lambda x: len(x.text), 
    batch_size = BATCH_SIZE, 
    device = device)



In [0]:
# for batch in test_iterator:
#       print(batch.text)

In [0]:

from transformers import BertTokenizer, BertModel
bert = BertModel.from_pretrained(bert_model)


In [0]:
import torch.nn as nn
import torch.nn.functional as F

class BERTGRUSentiment(nn.Module):
    def __init__(self,
                 bert,
                 hidden_dim,
                 output_dim,
                 n_layers,
                 bidirectional,
                 dropout):
        
        super().__init__()
        
        self.bert = bert
        
        embedding_dim = bert.config.to_dict()['hidden_size']
        # self.conv_0 = nn.Conv2d(in_channels = 1, 
        #                         out_channels = n_filters, 
        #                         kernel_size = (filter_sizes, embedding_dim))
        self.rnn = nn.GRU(embedding_dim,
                          hidden_dim,
                          num_layers = n_layers,
                          bidirectional = bidirectional,
                          batch_first = True,
                          dropout = 0 if n_layers < 2 else dropout)
        # self.conv = nn.Conv1d(74,74, 3)
        self.out = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [batch size, sent len]
                
        with torch.no_grad():
            embedded = self.bert(text)[0]
        #print(embedded.size())
        #embedded = [batch size, sent len, emb dim]
        
        _, hidden = self.rnn(embedded)
        #hidden = [n layers * n directions, batch size, emb dim]
        #print(hidden.size())
        if self.rnn.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        else:
            hidden = self.dropout(hidden[-1,:,:])
        #print(hidden.size())
        #hidden = [batch size, hid dim]
        # hidden = hidden.unsqueeze(1)
        output = self.out(hidden)
        
        #output = [batch size, out dim]
        
        return output
class BERTCNNSentiment(nn.Module):
    def __init__(self,
                 bert,
                 hidden_dim,
                 output_dim,
                 n_layers,
                 bidirectional,
                 dropout,n_filters,filter_sizes):
        
        super().__init__()
        
        self.bert = bert
        
        embedding_dim = bert.config.to_dict()['hidden_size']
        self.conv_0 = nn.Conv2d(in_channels = 1, 
                                out_channels = n_filters, 
                                kernel_size = (filter_sizes[0], embedding_dim))
        
        self.conv_1 = nn.Conv2d(in_channels = 1, 
                                out_channels = n_filters, 
                                kernel_size = (filter_sizes[1], embedding_dim))
        
        self.conv_2 = nn.Conv2d(in_channels = 1, 
                                out_channels = n_filters, 
                                kernel_size = (filter_sizes[2], embedding_dim))
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        
        self.dropout = nn.Dropout(dropout)

    def forward(self, text):
        
        #text = [batch size, sent len]
                
        with torch.no_grad():
            embedded = self.bert(text)[0]
        #print(embedded.size())
        #embedded = [batch size, sent len, emb dim]
        
        embedded = embedded.unsqueeze(1)
        
        #embedded = [batch size, 1, sent len, emb dim]
        
        conved_0 = F.relu(self.conv_0(embedded).squeeze(3))
        conved_1 = F.relu(self.conv_1(embedded).squeeze(3))
        conved_2 = F.relu(self.conv_2(embedded).squeeze(3))
            
        #conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]
        
        pooled_0 = F.max_pool1d(conved_0, conved_0.shape[2]).squeeze(2)
        pooled_1 = F.max_pool1d(conved_1, conved_1.shape[2]).squeeze(2)
        pooled_2 = F.max_pool1d(conved_2, conved_2.shape[2]).squeeze(2)
        
        #pooled_n = [batch size, n_filters]
        
        cat = self.dropout(torch.cat((pooled_0, pooled_1, pooled_2), dim = 1))

        #cat = [batch size, n_filters * len(filter_sizes)]
            
        return self.fc(cat)


In [33]:
HIDDEN_DIM = 100
OUTPUT_DIM = 3
N_LAYERS = 1
BIDIRECTIONAL = True
DROPOUT = 0.5
N_FILTERS = 100
FILTER_SIZES = [2,3,4]
model = BERTGRUSentiment(bert,
                         HIDDEN_DIM,
                         OUTPUT_DIM,
                         N_LAYERS,
                         BIDIRECTIONAL,
                         DROPOUT)

# model = BERTCNNSentiment(bert,
#                          HIDDEN_DIM,
#                          OUTPUT_DIM,
#                          N_LAYERS,
#                          BIDIRECTIONAL,
#                          DROPOUT,N_FILTERS,FILTER_SIZES)
# We can check how many parameters the model has. Our standard models have under 5M, but this one has 112M! Luckily, 110M of these parameters are from the transformer and we will not be training those.

# In[100]:


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')


# In order to freeze paramers (not train them) we need to set their `requires_grad` attribute to `False`. To do this, we simply loop through all of the `named_parameters` in our model and if they're a part of the `bert` transformer model, we set `requires_grad = False`. 

# In[101]:


# for name, param in model.named_parameters():                
#     if name.startswith('bert'):
#         param.requires_grad = False


# We can now see that our model has under 3M trainable parameters, making it almost comparable to the `FastText` model. However, the text still has to propagate through the transformer which causes training to take considerably longer.

# In[102]:


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')


The model has 522,603 trainable parameters
The model has 522,603 trainable parameters


In [0]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())


# In[105]:


criterion = nn.CrossEntropyLoss()


# Place the model and criterion onto the GPU (if available)

# In[106]:


model = model.to(device)
criterion = criterion.to(device)


# Next, we'll define functions for: calculating accuracy, performing a training epoch, performing an evaluation epoch and calculating how long a training/evaluation epoch takes.

# In[107]:




In [0]:

def categorical_accuracy(preds, y):
    count0,count1,count2 = torch.zeros(1),torch.zeros(1),torch.zeros(1)
    total0,total1,total2 = torch.FloatTensor(1),torch.FloatTensor(1),torch.FloatTensor(1)
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    correct = max_preds.squeeze(1).eq(y)
    true_correct = [0,0,0]
    for j,i in enumerate(y.cpu().numpy()):
      true_correct[y.cpu().numpy()[j]]+=1
      if i==0:
        count0+=correct[j]
        total0+=1
      elif i==1:
        count1+=correct[j]
        total1+=1
      elif i==2:
        count2+=correct[j]
      else:
        total2+=1

    return correct.sum() / torch.FloatTensor([y.shape[0]]),torch.FloatTensor([count0/true_correct[0],count1/true_correct[1],count2/true_correct[2]])


# In[108]:


def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)



        loss = criterion(predictions, batch.label)
        
        acc,_ = categorical_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    epoch_all_acc = torch.FloatTensor([0,0,0])
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc,all_acc = categorical_accuracy(predictions, batch.label)
            epoch_loss += loss.item()
            epoch_acc += acc.item()
            epoch_all_acc += all_acc
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator),epoch_all_acc/len(iterator)


# In[110]:


import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs


In [37]:
N_EPOCHS = 50

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc,tot = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), model_save_name)
        path = data_path+F"{model_save_name}"
        torch.save(model.state_dict(), path)
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')
    print(tot)


Epoch: 01 | Epoch Time: 1m 33s
	Train Loss: 1.015 | Train Acc: 46.80%
	 Val. Loss: 0.953 |  Val. Acc: 52.64%
tensor([0.3938, 0.6954, 0.4664])
Epoch: 02 | Epoch Time: 1m 32s
	Train Loss: 0.935 | Train Acc: 53.76%
	 Val. Loss: 0.908 |  Val. Acc: 56.53%
tensor([0.4299, 0.6043, 0.6590])
Epoch: 03 | Epoch Time: 1m 32s
	Train Loss: 0.893 | Train Acc: 56.55%
	 Val. Loss: 0.910 |  Val. Acc: 55.89%
tensor([0.3271, 0.7552, 0.5931])
Epoch: 04 | Epoch Time: 1m 32s
	Train Loss: 0.877 | Train Acc: 58.17%
	 Val. Loss: 0.897 |  Val. Acc: 56.90%
tensor([0.4663, 0.5268, 0.7204])
Epoch: 05 | Epoch Time: 1m 34s
	Train Loss: 0.859 | Train Acc: 59.61%
	 Val. Loss: 0.884 |  Val. Acc: 57.59%
tensor([0.4428, 0.5788, 0.7019])
Epoch: 06 | Epoch Time: 1m 33s
	Train Loss: 0.843 | Train Acc: 61.02%
	 Val. Loss: 0.870 |  Val. Acc: 58.17%
tensor([0.4934, 0.6042, 0.6252])
Epoch: 07 | Epoch Time: 1m 33s
	Train Loss: 0.812 | Train Acc: 62.50%
	 Val. Loss: 0.879 |  Val. Acc: 57.73%
tensor([0.3686, 0.6795, 0.6985])
Epoch:

In [0]:
path = data_path+F"{model_save_name}"
model.load_state_dict(torch.load(path))

In [0]:

def test_evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    epoch_all_acc = torch.FloatTensor([0,0,0])
    print(epoch_all_acc)
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc,all_acc = test_categorical_accuracy(predictions, batch.label)
            print(all_acc)
            epoch_loss += loss.item()
            epoch_acc += acc.item()
            epoch_all_acc += all_acc
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator),epoch_all_acc/len(iterator)

def test_categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    count0,count1,count2 = torch.zeros(1),torch.zeros(1),torch.zeros(1)
    total0,total1,total2 = torch.FloatTensor(1),torch.FloatTensor(1),torch.FloatTensor(1)
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    correct = max_preds.squeeze(1).eq(y)
    true_correct = [0,0,0]
    for j,i in enumerate(y.cpu().numpy()):
      true_correct[y.cpu().numpy()[j]]+=1
      if i==0:
        count0+=correct[j]
        total0+=1
      elif i==1:
        count1+=correct[j]
        total1+=1
      elif i==2:
        count2+=correct[j]
      else:
        print(i,i==0,i==1,i==2)
        total2+=1
    # print(count0,count1,count2,total0,total1,total2)
    # print([count0/total0,count1/total1,count2/total2])
    # print(torch.FloatTensor([count0/total0,count1/total1,count2/total2]))
    # print(correct.sum() / torch.FloatTensor([y.shape[0]]))
    # print(torch.FloatTensor([count0/total0,count1/total1,count2/total2]))
    print(count0,count1,count2)
    return correct.sum() / torch.FloatTensor([y.shape[0]]),torch.FloatTensor([count0/true_correct[0],count1/true_correct[1],count2/true_correct[2]])



In [0]:
# model.load_state_dict(torch.load('tut6-model.pt'))

test_loss, test_acc,test_all_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%',test_all_acc)


# ## Inference
# 
# We'll then use the model to test the sentiment of some sequences. We tokenize the input sequence, trim it down to the maximum length, add the special tokens to either side, convert it to a tensor, add a fake batch dimension and then pass it through our model.

# In[ ]:







In [0]:
print(len(test_iterator))
24*128

In [0]:
print(len(test_data))

In [0]:
# {'1': 0, '2': 1, '0': 2})
m = {0:'neutral',1:'positive',2:'negative'}
file = open('answer.txt','w')
file.write('Uid,Sentiment\n')
count = 0
for batch in test_iterator:
  predictions = model(batch.text).squeeze(1)
  max_preds = predictions.argmax(dim = 1, keepdim = True).detach().cpu().numpy()
  # print(batch.uid)
  # print(max_preds)
  for i,row in enumerate(batch.uid.cpu().numpy()):
    count += 1
    print(count)
    if count != len(test_data):
      file.write('%s,%s\n'%(row,m[max_preds[i][0]]))
    else:
      file.write('%s,%s'%(row,m[max_preds[i][0]]))
    # if count != len(test_data):
    #   file.write('%s,%s,%s,%s\n'%(row," ".join(tokenizer.convert_)m[max_preds[i][0]]))
    # else:
    #   file.write('%s,%s'%(row,m[max_preds[i][0]]))
print(count)
file.close()

In [0]:
len(l)
l


In [0]:
f = open('')
for i in l:


In [0]:
def predict_sentiment(model, tokenizer, sentence):
    model.eval()
    tokens = tokenizer.tokenize(sentence)
    tokens = tokens[:max_input_length-2]
    indexed = [init_token_idx] + tokenizer.convert_tokens_to_ids(tokens) + [eos_token_idx]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(0)
    prediction = torch.sigmoid(model(tensor))
    # ind = np.argmax(np.array(prediction))
    # if ind ==0:
    #   print('neutral')
    # elif ind == 1:
    #   print("positive")
    # else:
    #   print("negative")
    print(prediction)
predict_sentiment(model, tokenizer, "This film is terrible")


# In[ ]:


predict_sentiment(model, tokenizer, "This film is great")

In [0]:
while True:
  sent = input('->')
  if sent != '$':
    predict_sentiment(model, tokenizer, sent)
  else:
    break
